In [1]:
import time
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import configparser
config = configparser.ConfigParser()
config.read("config.ini")

['config.ini']

['config.ini']

In [2]:
# Udemy 登入資訊
email = config.get("udemy", "email")
password = config.get("udemy", "password")

In [3]:
# 抓取筆記的課程 URL
course_url = config.get("udemy", "course_url")

In [4]:
# 建立瀏覽器實例
options = uc.ChromeOptions()

options.add_argument("--disable-blink-features")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
# options.add_argument("--lang=zh-TW")  # 繁體中文界面


driver = uc.Chrome(executable_path='./chromedriver.exe', options=options)


driver.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {'source': """
    Object.defineProperty(Navigator.prototype, 'webdriver', {
        set: undefined,
        enumerable: true,
        configurable: true,
        get: new Proxy(
            Object.getOwnPropertyDescriptor(Navigator.prototype, 'webdriver').get,
            { apply: (target, thisArg, args) => {
                // emulate getter call validation
                Reflect.apply(target, thisArg, args);
                return false;
            }}
        )
    });
"""})


# 登入 Udemy
driver.get("https://www.udemy.com/join/login-popup/")

In [5]:
# 前往課程頁面
try:
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".ud-btn.ud-btn-large.ud-btn-brand.ud-heading-md.helpers--auth-submit-button--2K2dh")))
    print("頁面載入成功")
    
    driver.find_element(By.NAME, "email").send_keys(email)
    driver.find_element(By.NAME, "password").send_keys(password)
except:
    print("未能在指定時間內載入頁面")

頁面載入成功
頁面載入成功


In [6]:
# driver.find_element(By.NAME, "submit").click()
driver.find_element(By.CSS_SELECTOR, ".ud-btn.ud-btn-large.ud-btn-brand.ud-heading-md.helpers--auth-submit-button--2K2dh").click()

In [7]:
# 等待登入成功
try:
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "span[class='ud-text-sm header--dropdown-button-text--2OxOV']")))
    print("登入成功")
except:
    print("未能在指定時間內找到元素，但程式將繼續執行。")

登入成功
登入成功


In [8]:
# 前往課程頁面
try:
    time.sleep(3)
    driver.get(course_url)
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "button#dropdown-trigger--3")))
    print("前往課程頁面成功")
except:
    print("未能在指定時間內前往課程頁面")

前往課程頁面成功
前往課程頁面成功


In [9]:
# 選擇所有講座
time.sleep(3)

lecture_dropdown_trigger = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button#dropdown-trigger--3")))
lecture_dropdown_trigger.click()

all_lectures_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-purpose='filter-by-all-lectures']")))
all_lectures_button.click()

# 依最舊至最新排序
sort_dropdown_trigger = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button#dropdown-trigger--6")))
sort_dropdown_trigger.click()

oldest_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-purpose='sort-by-oldest']")))
oldest_button.click()

time.sleep(3)

In [10]:
try:
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-purpose='bookmarks-container']")))
    
    notes_container = driver.find_element(By.CSS_SELECTOR, "div[data-purpose='bookmarks-container']")
    notes = notes_container.find_elements(By.CSS_SELECTOR, "div.lecture-bookmark-v2--row--1JALI")

    with open("notes_output.txt", "w", encoding="utf-8") as output_file:
        for note in notes:
            section = note.find_element(By.CSS_SELECTOR, "div.lecture-bookmark-v2--section--383LP").text
            title = note.find_element(By.CSS_SELECTOR, "div.ud-text-sm").text
            content = note.find_element(By.CSS_SELECTOR, "div[data-purpose='safely-set-inner-html:rich-text-viewer:html']").text

            output_file.write(f"Section: {section}\n")
            output_file.write(f"Title: {title}\n")
            output_file.write(f"Content: {content}\n")
            output_file.write("\n---\n")

finally:
#     driver.quit()
    print('筆記輸出成功')

筆記輸出成功
筆記輸出成功
